In [1]:
import os,sys, re
from importlib import reload
import numpy as np
import pandas as pd
from Bio.Seq import reverse_complement
#import torch
print(os.getpid())
sys.path.append(r"/lab/weissman_imaging/puzheng/Softwares")

from ChromAn.src.library_tools.primers import load_fasta_to_DataFrame
#import MERFISH_probe_design.probe_design.OTTable_dict as ot

1025419


In [2]:
from ChromAn.src.file_io.data_organization import Color_Usage, search_fovs_in_folders, color_usage_kwds

In [3]:
import ChromAn
reload(ChromAn.src.file_io.data_organization)
from ChromAn.src.file_io.data_organization import Data_Organization, search_fovs_in_folders
from ChromAn.src.file_io.dax_process import DaxProcesser
from ChromAn.src.visual_tools.interactive import imshow_mark_3d

In [5]:
ls /lab/weissman_imaging/puzheng/4T1Tumor

 20230625-4T1cell_c2/
 20230821-F133section18_PFAfix_dapi-mcherry/
 20231226-MF4DNA_4T1preEditTop8/
 20240109-IF_F170/
 20240128-F168_conA/
 20240128-F168_Phalloidin/
 20240128-oligo_IF_before_clearing/
 20240206-F168_antiEPCAM/
 20240214-F168_epCAM/
 20240229-F235_epCAM_perfused/
 20240229-F236_epCAM/
 20240327-F133_mcherry/
 20240327-F235_mcherry/
 20240331-4T1fullyEdited_MF4dna/
 20240401-4T1fullyEdited_2gene-10nM/
 20240401-F235fullyEdited_2gene/
 20240402-4T1fullyEdited_2gene-1nM/
 20240419-F240fresh_Cldn4-Fscn1-mCherry/
 20240419-F242dpec_Cldn4-Fscn1-mCherry/
 20240419-fullyEdit4T1_smFISH-Cldn4-Fscn1-mCherry/
 20240422-F244pfa_smFISH_mCherry-Cldn4-Fscn1/
 20240423-F242depc_MF4dna/
 20240423-F242dpec_smFISH_mCherry-Cldn4-Fscn1/
 20240504-fullyEdited4T1_MF8/
 20240505-4T1fullyEdited_MF8_bsaIF/
 20240505-F242depc_MF4dna_IF/
 20240506-4T1fullyEdited_MF8_fcblockIF/
 20240510-F242depc20um_MF4dna/
 20240511-F33212um_MF4dna_mcherry/
 20240512-F242depc10um_MF4dna_mcherry/
 20240519-4T1noM

In [6]:
# where the original data saves
data_folder =  r"/lab/weissman_imaging/puzheng/4T1Tumor/20240605-F319_depcTailVein_MF4dna_mcherry"
#data_folder = os.path.join(data_home, '4T1Tumor', '20240504-fullyEdited4T1_MF8')

experiment_date = os.path.basename(data_folder).split('-')[0]
print(experiment_date)
# Color usage file marks the organization of imaging files
color_usage_filename = os.path.join(data_folder, 'Analysis', 'color_usage_MF4dna.csv')
print(os.path.exists(color_usage_filename))

20240605
True


In [7]:
# Load readout names:
library_folder = r'/lab/solexa_weissman/puzheng/MERFISH_Probes/4T1_MERFISH/MF4_4T1Primary16bit_DNAbit'
readout_filename = os.path.join(library_folder, 'readouts.csv')

if os.path.exists(readout_filename):
    readouts = pd.read_csv(readout_filename) 
else:
    # generate readout_file
    import MERFISH_probe_design.IO.file_io as fio
    merfish_bits_file = os.path.join(r'/lab/solexa_weissman/puzheng/References/Readouts', 'MERFISH_readouts.fasta')
    merfish_bits = fio.load_fasta_into_df(merfish_bits_file)
    # first 22 bits are for this library:
    readouts = {'Name':[_id.split('-')[-1] for _id in merfish_bits.loc[:, 'id']], 
                'Sequence':merfish_bits.loc[:, 'sequence']}
    readouts = pd.DataFrame(readouts)
    # save this readout
    readouts.to_csv(readout_filename, index=None)
# select names
readout_names = list(readouts['Name'])
print(len(readout_names), readout_names)

16 ['Stv_29', 'Stv_30', 'Stv_31', 'Stv_32', 'Stv_33', 'Stv_35', 'Stv_36', 'Stv_37', 'Stv_39', 'Stv_40', 'Stv_42', 'Stv_44', 'Stv_45', 'Stv_46', 'Stv_48', 'Stv_50']


In [8]:
# Generate data organization
data_organization_folder = r'/lab/weissman_imaging/puzheng/Softwares/Weissman_MERFISH_Scripts/merlin_parameters/dataorganization'

data_organization_filename = os.path.join(data_organization_folder, 
                                          f'{experiment_date}-{os.path.basename(library_folder).split("_")[0]}_{len(readout_names)}bit.csv')

_do = Data_Organization(data_organization_filename,)
if _do.is_empty():
    # create
    _do.create_from_colorUsage(color_usage_filename, data_folder, None, readout_names=readout_names, reorganized=False)
    # save
    _do.save_to_file(overwrite=True)

- load color_usage from file: /lab/weissman_imaging/puzheng/4T1Tumor/20240605-F319_depcTailVein_MF4dna_mcherry/Analysis/color_usage_MF4dna.csv
- searching in folder: /lab/weissman_imaging/puzheng/4T1Tumor/20240605-F319_depcTailVein_MF4dna_mcherry
-- 9 folders, 178 fovs detected.
MERFISHbit 0 bit-1 748 H0M1
MERFISHbit 1 bit-2 637 H0M1
MERFISHbit 2 bit-3 748 H1M2
MERFISHbit 3 bit-4 637 H1M2
MERFISHbit 4 bit-5 748 H2M3
MERFISHbit 5 bit-6 637 H2M3
MERFISHbit 6 bit-7 748 H3M4
MERFISHbit 7 bit-8 637 H3M4
MERFISHbit 8 bit-9 748 H4M5
MERFISHbit 9 bit-10 637 H4M5
MERFISHbit 10 bit-11 748 H5M6
MERFISHbit 11 bit-12 637 H5M6
MERFISHbit 12 bit-13 748 H6M7
MERFISHbit 13 bit-14 637 H6M7
MERFISHbit 14 bit-15 748 H7M8
MERFISHbit 15 bit-16 637 H7M8
- 16 MERFISH rows appended.
Other info: DAPI 405 H0M1
Other info: mcherry1 748 H8M9
Other info: mcherry2 637 H8M9
Saving data_organization into file: /lab/weissman_imaging/puzheng/Softwares/Weissman_MERFISH_Scripts/merlin_parameters/dataorganization/20240605-

In [9]:
_do

,channelName,readoutName,imageType,imageRegExp,bitNumber,imagingRound,color,frame,zPos,fiducialImageType,fiducialRegExp,fiducialImagingRound,fiducialFrame,fiducialColor
0,bit1,Stv_29,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,1,0,748,[0 4 8 12 16 20 24 28 32 36 40 44 48 52 56 60 ...,[-12.0 -10.8 -9.6 -8.4 -7.2 -6.0 -4.8 -3.6 -2....,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,0,[2 6 10 14 18 22 26 30 34 38 42 46 50 54 58 62...,477
1,bit2,Stv_30,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,2,0,637,[1 5 9 13 17 21 25 29 33 37 41 45 49 53 57 61 ...,[-12.0 -10.8 -9.6 -8.4 -7.2 -6.0 -4.8 -3.6 -2....,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,0,[2 6 10 14 18 22 26 30 34 38 42 46 50 54 58 62...,477
2,bit3,Stv_31,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,3,1,748,[0 3 6 9 12 15 18 21 24 27 30 33 36 39 42 45 4...,[-12.0 -10.8 -9.6 -8.4 -7.2 -6.0 -4.8 -3.6 -2....,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,1,[2 5 8 11 14 17 20 23 26 29 32 35 38 41 44 47 ...,477
3,bit4,Stv_32,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,4,1,637,[1 4 7 10 13 16 19 22 25 28 31 34 37 40 43 46 ...,[-12.0 -10.8 -9.6 -8.4 -7.2 -6.0 -4.8 -3.6 -2....,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,1,[2 5 8 11 14 17 20 23 26 29 32 35 38 41 44 47 ...,477
4,bit5,Stv_33,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,5,2,748,[0 3 6 9 12 15 18 21 24 27 30 33 36 39 42 45 4...,[-12.0 -10.8 -9.6 -8.4 -7.2 -6.0 -4.8 -3.6 -2....,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,2,[2 5 8 11 14 17 20 23 26 29 32 35 38 41 44 47 ...,477
5,bit6,Stv_35,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,6,2,637,[1 4 7 10 13 16 19 22 25 28 31 34 37 40 43 46 ...,[-12.0 -10.8 -9.6 -8.4 -7.2 -6.0 -4.8 -3.6 -2....,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,2,[2 5 8 11 14 17 20 23 26 29 32 35 38 41 44 47 ...,477
6,bit7,Stv_36,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,7,3,748,[0 3 6 9 12 15 18 21 24 27 30 33 36 39 42 45 4...,[-12.0 -10.8 -9.6 -8.4 -7.2 -6.0 -4.8 -3.6 -2....,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,3,[2 5 8 11 14 17 20 23 26 29 32 35 38 41 44 47 ...,477
7,bit8,Stv_37,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,8,3,637,[1 4 7 10 13 16 19 22 25 28 31 34 37 40 43 46 ...,[-12.0 -10.8 -9.6 -8.4 -7.2 -6.0 -4.8 -3.6 -2....,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,3,[2 5 8 11 14 17 20 23 26 29 32 35 38 41 44 47 ...,477
8,bit9,Stv_39,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,9,4,748,[0 3 6 9 12 15 18 21 24 27 30 33 36 39 42 45 4...,[-12.0 -10.8 -9.6 -8.4 -7.2 -6.0 -4.8 -3.6 -2....,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,4,[2 5 8 11 14 17 20 23 26 29 32 35 38 41 44 47 ...,477
9,bit10,Stv_40,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,10,4,637,[1 4 7 10 13 16 19 22 25 28 31 34 37 40 43 46 ...,[-12.0 -10.8 -9.6 -8.4 -7.2 -6.0 -4.8 -3.6 -2....,Conv_zscan,H(?P<imagingRound>[0-9]+)M([0-9]+)/(?P<imageTy...,4,[2 5 8 11 14 17 20 23 26 29 32 35 38 41 44 47 ...,477


## Generate position file
NEW: loop through first round and generate the position_file:

In [10]:
from ChromAn.src.file_io.dax_process import DaxProcesser

folders, fovs = search_fovs_in_folders(data_folder)

reference_folder = folders[0]
positions = []

for _fov in fovs:
    daxp = DaxProcesser(os.path.join(reference_folder, _fov), verbose=False)
    positions.append(daxp._FindGlobalPosition(daxp.filename))
positions = np.array(positions)

- searching in folder: /lab/weissman_imaging/puzheng/4T1Tumor/20240605-F319_depcTailVein_MF4dna_mcherry
-- 9 folders, 178 fovs detected.


In [11]:
position_folder = r"/lab/weissman_imaging/puzheng/Softwares/Weissman_MERFISH_Scripts/merlin_parameters/positions"
target_position_filename = os.path.join(position_folder, f"{experiment_date}_positions.txt")
print(f"target position file: {target_position_filename}")
np.savetxt(target_position_filename, positions, fmt='%.2f', delimiter=',')

target position file: /lab/weissman_imaging/puzheng/Softwares/Weissman_MERFISH_Scripts/merlin_parameters/positions/20240605_positions.txt
